# Script de création d'un geojson complet pour les bassins d'emploi

## Contexte

Afin de réaliser certaines représentations cartographiques, il est nécessaire de disposer de fonds de cartes, notamment pour représenter les limites de régions. Le site https://france-geojson.gregoiredavid.fr/ propose des fonds de carte en GeoJSON très faciles à utiliser avec la bibliothèque `plotly`. Nous avons ainsi pu réaliser des représentations en fonction des régions ou des départements. Toutefois, nous avons constaté que ce site ne propose pas de fond de carte pour les bassins d'emploi. Il convient donc de trouver une autre source de données cartographiques pour cette représentation spécifique.

Après des recherches approfondies, nous n'avons pas réussi à trouver des fonds de carte pour les bassins d'emploi autres que ceux proposés par l'INSEE. Toutefois, les fonds de carte de l'INSEE ne correspondent pas exactement au découpage des bassins d'emploi de Pôle emploi, qui en compte environ 400 contre 300 pour l'INSEE. Par conséquent, nous n'avons pas pu les utiliser.
</br>
En fouillant un peu sur la source des données, nous avons découvert le site https://dataemploi.pole-emploi.fr/accueil, qui permet de visualiser certaines données en fonction d'un territoire donné, tel qu'un bassin d'emploi. En outre, la carte interactive proposée par ce site affiche clairement les limites des bassins d'emploi. </br>
Par chance, nous avons remarqué en examinant l'activité réseau (sur chrome F12 -> onglet network) que lorsqu'on demande à voir la carte, une requête est effectuée vers l'URL https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=%CODE_REG%&isbassin=true, qui renvoie un fichier GeoJSON contenant les fonds de carte des bassins d'emploi d'une région donnée.



Le script suivant va donc effectuer une requête à l'URL https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=%CODE_REG%&isbassin=true pour chaque région, puis rassembler les fichiers GeoJSON obtenus en un seul. Cela nous permettra de réaliser des représentations cartographiques en fonction des bassins d'emploi.

## Script

In [1]:
import requests,json
import pandas as pd
import sqlite3

On récupère les numéros des régions :

In [2]:
db = sqlite3.connect("Emploi.db")
nreg=pd.read_sql_query("SELECT reg FROM geo",db)["reg"].unique()

Pour chaque region, on fait des requêtes aux serveurs de pôle emploi pour avoir le geojson des bassins d'emploi. Enfin, on concatene ces geojson pour obtenir un fichier geojson contenant les bassins d'emploi de toute la France : 

In [3]:
geo_json={'type': 'FeatureCollection','name': 'Bassins-Emploi-Région','features':[]}
for n in nreg:
    code='{0:0>2}'.format(n)
    response = requests.get(f"https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t={code}&isbassin=true")
    print(f"https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t={code}&isbassin=true   RESPONSE : {response.text if len(response.text)<50 else 'GOOD'}")
    js=json.loads(response.text)
    geo_json["features"]+=js["features"]

https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=84&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=32&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=93&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=44&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=76&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=28&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=75&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=24&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=27&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=53&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=94&isbassin=true   RESPONSE : GOOD
https://dataemploi.pole-emploi.fr/geoJson?tt=REG&t=52&

Ici on exporte simplement ce geojson dans un fichier intitulé bassins.geojson 

In [5]:
with open("data/geojson/bassins.geojson", "w",encoding="utf8") as outfile:
    json.dump(geo_json, outfile,ensure_ascii=False)